In [8]:
import zipfile
import os
from PIL import Image
from tqdm import tqdm  # Optional: for progress bar

# Step 1: Extract the zip file
zip_file_path = 'train_data.zip'
extract_folder = 'extracted_train_data'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Step 2: Label the data
train_data = []

for digit in range(10):
    digit_folder = os.path.join(extract_folder, str(digit))

    for image_file in tqdm(os.listdir(digit_folder), desc=f'Labeling digit {digit}'):
        image_path = os.path.join(digit_folder, image_file)
        train_data.append((image_path, digit))

# Print the first few entries to verify the labeling
print("Labeled Data Samples:")
for sample in train_data[:5]:
    print(sample)


Labeling digit 9: 100%|██████████| 80/80 [00:00<00:00, 301748.49it/s]

Labeled Data Samples:
('extracted_train_data/0/0_63.jpg', 0)
('extracted_train_data/0/0_69.jpg', 0)
('extracted_train_data/0/0_37.jpg', 0)
('extracted_train_data/0/0_18.jpg', 0)
('extracted_train_data/0/0_29.jpg', 0)


In [9]:
import zipfile
import os
from PIL import Image
from tqdm import tqdm  # Optional: for progress bar

# Step 1: Extract the zip file
zip_file_path = 'test_data.zip'
extract_folder = 'extracted_test_data'

with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)

# Step 2: Label the data
test_data = []

for digit in range(10):
    digit_folder = os.path.join(extract_folder, str(digit))

    for image_file in tqdm(os.listdir(digit_folder), desc=f'Labeling digit {digit}'):
        image_path = os.path.join(digit_folder, image_file)
        test_data.append((image_path, digit))

# Print the first few entries to verify the labeling
print("Labeled Data Samples:")
for sample in test_data[:5]:
    print(sample)


Labeling digit 9: 100%|██████████| 20/20 [00:00<00:00, 165130.08it/s]

Labeled Data Samples:
('extracted_test_data/0/0_83.jpg', 0)
('extracted_test_data/0/0_85.jpg', 0)
('extracted_test_data/0/0_87.jpg', 0)
('extracted_test_data/0/0_98.jpg', 0)
('extracted_test_data/0/0_91.jpg', 0)


In [12]:
import torch
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(f"We are training on {device} for this experiment")

assert 'cuda' in repr(device), "GPU is not selected in hardware accelerator dropdown"

We are training on cuda:0 for this experiment


In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
%matplotlib inline
import torch
import torchvision
import torchvision.transforms as transforms

In [15]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image

# Assuming labeled_data is your list of tuples
class CustomDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = data
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path, label = self.data[idx]
        image = Image.open(image_path).convert('L')  # Convert to grayscale

        if self.transform:
            image = self.transform(image)

        return image, label

# Define transformations
transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Create custom dataset
custom_dataset1 = CustomDataset(data=train_data, transform=transform)
custom_dataset2 = CustomDataset(data=test_data, transform=transform)

# Create DataLoader
batch_size = 4
shuffle = True
num_workers = 2

custom_loader1 = DataLoader(custom_dataset1, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)
custom_loader2 = DataLoader(custom_dataset2, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [17]:
import torch.nn as nn
import torch.nn.functional as F

#Define your own 'resnet9 without residual connections' neural network class with name 'Net' here
#Note :
######## Thou shall not use the following commented out Net() definition in final evaluation
######## This is placed here for you to play with incase your model class definition doesn't work

'''
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
'''

def conv_block(in_channels, out_channels, pool=False):
    layers = [
                nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
    ]
    if pool: layers.append(nn.MaxPool2d(2))
    return nn.Sequential(*layers)

class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.conv1 = conv_block(1, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))

        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))

        self.classifier = nn.Sequential(nn.MaxPool2d(2),
                                        nn.Flatten(),
                                        nn.Dropout(0.2),
                                        nn.Linear(512, 10),
                                        nn.Softmax(dim=1))
    def forward(self,x):
        out=self.conv1(x)
        out=self.conv2(out)
        out=self.res1(out)+out
        out=self.conv3(out)
        out=self.conv4(out)
        out=self.res2(out)+out
        out=self.classifier(out)
        return out

model_params_file_path = "/content/drive/MyDrive/checkpoints/best_model.pth"

net = Net().to(device)
checkpoint = torch.load(model_params_file_path, map_location=torch.device('cpu'))  # Adjust map_location if using GPU
net.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [18]:
#Freeze the parameters
for param in net.parameters():
    param.requires_grad = False

# Add a fully connected layer at the end
num_ftrs = net.classifier[3].in_features
net.classifier[3] = nn.Linear(num_ftrs, 10)

In [19]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.classifier[3].parameters(), lr=0.001, momentum=0.9)

In [43]:
def validator(testloader=None,net=None):
    correct = 0
    total = 0
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for data in custom_loader2:
            images, labels = data
            images = images.to(device)
            labels = labels.to(device)

            # calculate outputs by running images through the network
            outputs = net(images)
            # the class with the highest energy is what we choose as prediction
            # perform max along dimension 1, since dimension 0 is batch dimension
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Accuracy of the network on the 200 test images: {100 * correct // total} %')
    return correct/total

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:
your_google_drive_path = "/content/drive/MyDrive/checkpoints/"
import os
assert os.path.isdir(your_google_drive_path), f"{your_google_drive_path} is not a valid location"

In [44]:
start_epoch=0
end_epoch = 10

best_accuracy = -1.0
for epoch in range(start_epoch,end_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(custom_loader1, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        net = net. to(device)
        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch: {epoch}')
    current_accuracy = validator(testloader=custom_loader2,net=net)
    if current_accuracy>best_accuracy:
        best_accuracy = current_accuracy

        torch.save(
            {'epoch':epoch,
             'model_state_dict': net.state_dict(),
             'optimizer_state_dict': optimizer.state_dict()
             },

             your_google_drive_path+'best_model4.pth')

    #Save model as checkpoint
    torch.save(
        {'epoch':epoch,
         'model_state_dict': net.state_dict(),
         'optimizer_state_dict': optimizer.state_dict()
         },
         your_google_drive_path+'checkpoint4.pth')



print('Finished Training')

Epoch: 0
Accuracy of the network on the 200 test images: 67 %
Epoch: 1
Accuracy of the network on the 200 test images: 60 %
Epoch: 2
Accuracy of the network on the 200 test images: 66 %
Epoch: 3
Accuracy of the network on the 200 test images: 69 %
Epoch: 4
Accuracy of the network on the 200 test images: 64 %
Epoch: 5
Accuracy of the network on the 200 test images: 67 %
Epoch: 6
Accuracy of the network on the 200 test images: 66 %
Epoch: 7
Accuracy of the network on the 200 test images: 65 %
Epoch: 8
Accuracy of the network on the 200 test images: 70 %
Epoch: 9
Accuracy of the network on the 200 test images: 63 %
Finished Training
